In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from load import dataset

In [16]:
# Initialize Variables
predict_week = 3
predict_season = 2020
today_localize = pd.to_datetime('today').tz_localize('US/Central') - pd.Timedelta(hours=3)

final_columns = ['game_id', 'season', 'week', 'season_type', 'home_team',
                'home_conference', 'away_team', 'away_conference',  'pre_game_spread',
                'predicted_spread', 'regression_spread_pred', 'classification_spread_pred', 'classification_confidence']

In [17]:
# Load Dataset
df = dataset(predict_season=predict_season, predict_week=predict_week, window_size=4, update_data=True, update_seasons=[pd.to_datetime('today').year])

# Prep for ML Models
df[df.columns[df.columns.str.contains('home|Home')]] = df[df.columns[df.columns.str.contains(
    'home|Home')]].groupby('home_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df[df.columns[df.columns.str.contains('away|Away')]] = df[df.columns[df.columns.str.contains(
    'away|Away')]].groupby('away_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

df = df.dropna(axis=1, thresh=int(len(df) * .9))

df = df.fillna(0)

df[df.select_dtypes('bool').columns] = df.select_dtypes('bool').astype('int')

ct = make_column_transformer(
    (OrdinalEncoder(), ['season_type', 'home_conference', 'away_conference']),
    remainder='passthrough'
)

drop_columns =  ['game_id', 'season', 'venue', 'start_date', 'home_team', 'away_team', 
                'start_date', 'spread_target', 'pre_game_spread' , 'pre_game_home_win_prob', 'pre_game_away_win_prob'
                ]


In [18]:
# Use in production
X_train = df[(df.start_date.dt.tz_convert('US/Central') < today_localize)]
X_test = df[(df.start_date.dt.tz_convert('US/Central') >= today_localize)] 

# Use in development
# X_train = df[(df['week'] < predict_week) | (df['season'] < predict_season)] 
# X_test = df[(df['week'] == predict_week) & (df['season'] == predict_season)] 

model_regressor = XGBRegressor(learning_rate=0.1, 
                                colsample_bytree=0.9,
                                gamma=0.5,
                                max_depth=2,
                                min_child_weight=4,
                                n_estimators=100,
                                subsample=0.8)

model_regressor.fit(
    ct.fit_transform(
        X_train.drop(drop_columns, axis=1)), 
        X_train['spread_target']
        )

X_train['predicted_spread'] = model_regressor.predict(ct.fit_transform(X_train.drop(drop_columns, axis=1)))
X_test['predicted_spread'] = model_regressor.predict(ct.fit_transform(X_test.drop(drop_columns, axis=1)))

# Clean Up
X_train['predicted_spread'] = np.around(X_train['predicted_spread'] / .5, decimals=0) * .5
X_test['predicted_spread'] = np.around(X_test['predicted_spread'] / .5, decimals=0) * .5

X_train['regression_spread_pred'] = np.where(X_train['predicted_spread'] <= X_train['pre_game_spread'], 1, 0)
X_test['regression_spread_pred'] = np.where(X_test['predicted_spread'] <= X_test['pre_game_spread'], 1, 0)

X_train['spread_result'] = np.where(X_train['spread_target'] <= X_train['pre_game_spread'], 1, 0)
X_test['spread_result'] = np.where(X_test['spread_target'] <= X_test['pre_game_spread'], 1, 0)

model_classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
            colsample_bynode=1, colsample_bytree=0.8, gamma=5, gpu_id=-1,
            importance_type='gain', interaction_constraints='',
            learning_rate=0.1, max_delta_step=0, max_depth=5,
            min_child_weight=10, monotone_constraints='()',
            n_estimators=100, n_jobs=0, num_parallel_tree=1,
            objective='binary:logistic', random_state=0, reg_alpha=0,
            reg_lambda=1, scale_pos_weight=1, subsample=1.0,
            tree_method='exact', validate_parameters=1, verbosity=None)

# Update drop columns list for Classifier Model
drop_columns.append('spread_result')
# drop_columns.remove('pre_game_spread')   

model_classifier.fit(ct.fit_transform(X_train.drop(drop_columns, axis=1)), X_train['spread_result'])

X_train['classification_spread_pred'] = model_classifier.predict(ct.fit_transform(X_train.drop(drop_columns, axis=1)))
X_test['classification_spread_pred'] = model_classifier.predict(ct.fit_transform(X_test.drop(drop_columns, axis=1)))

# Add so the Predict Probability is Aligned
drop_columns.append('classification_spread_pred')

X_train['classification_confidence'] = model_classifier.predict_proba(ct.fit_transform(X_train.drop(drop_columns, axis=1)))[:,0]
X_test['classification_confidence'] = model_classifier.predict_proba(ct.fit_transform(X_test.drop(drop_columns, axis=1)))[:,0] 


In [19]:
predict_df = X_test[final_columns]
predict_df.head(10)

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,regression_spread_pred,classification_spread_pred,classification_confidence
5608,401234571,2020,3,regular,Pittsburgh,ACC,Syracuse,ACC,-21.0,-15.0,0,0,0.532774
5609,401234567,2020,3,regular,Duke,ACC,Boston College,ACC,-6.0,2.5,0,1,0.403426
5610,401207124,2020,3,regular,Western Kentucky,Conference USA,Liberty,FBS Independents,-14.0,-6.0,0,0,0.654454
5611,401212522,2020,3,regular,Tulane,American Athletic,Navy,American Athletic,-7.0,-10.5,1,1,0.243821
5612,401245593,2020,3,regular,Baylor,Big 12,Houston,American Athletic,-3.5,-0.5,0,0,0.748068
5613,401236041,2020,2,regular,Oklahoma State,Big 12,Tulsa,American Athletic,-24.0,-5.0,0,1,0.275504
5614,401241282,2020,3,regular,Notre Dame,FBS Independents,South Florida,American Athletic,-24.0,-20.5,0,0,0.746482
5615,401235433,2020,3,regular,North Carolina,ACC,Charlotte,Conference USA,-29.5,-21.5,0,1,0.357010
5616,401240640,2020,3,regular,Middle Tennessee,Conference USA,Troy,Sun Belt,3.0,-6.5,1,1,0.322234
5617,401234568,2020,3,regular,Georgia Tech,ACC,UCF,American Athletic,7.5,6.0,1,1,0.332734


In [20]:
# Pull in games that have occurred within the current week to append to the predictions
filepath = Path('../zillion_picks/picks')

result_dfs = []

for file in filepath.rglob('*.csv'):
    result_df = pd.read_csv(file)


    result_dfs.append(result_df)

result_df = pd.concat(result_dfs)

result_df = result_df[(result_df['week'] == predict_week) & (result_df['season'] == predict_season)]

try:
    result_df = result_df[~result_df.game_id.isin(predict_df.game_id)]
    result_df = result_df.drop(['actual_spread', 'spread_result', 'start_date'], axis=1)

except:
        result_df = result_df.drop(['actual_spread', 'spread_result', 'start_date'], axis=1)

result_df.head()

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,regression_spread_pred,classification_spread_pred,classification_confidence,regression_spread_result,classification_spread_result


In [21]:
# Merge Predit and Result Dataframes
try:
    predict_df = pd.concat([result_df, predict_df])
except:
    predict_df = result_df

In [22]:
# Add spread target in case the game is complete
predict_df = predict_df.merge(df[['game_id', 'spread_target', 'start_date']], on='game_id', how='left')
predict_df.head()

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,regression_spread_pred,classification_spread_pred,classification_confidence,regression_spread_result,classification_spread_result,spread_target,start_date
0,401234571,2020,3,regular,Pittsburgh,ACC,Syracuse,ACC,-21.0,-15.0,0,0,0.532774,NaN,NaN,0.0,2020-09-19 16:00:00+00:00
1,401234567,2020,3,regular,Duke,ACC,Boston College,ACC,-6.0,2.5,0,1,0.403426,NaN,NaN,0.0,2020-09-19 16:00:00+00:00
2,401207124,2020,3,regular,Western Kentucky,Conference USA,Liberty,FBS Independents,-14.0,-6.0,0,0,0.654454,NaN,NaN,0.0,2020-09-19 16:00:00+00:00
3,401212522,2020,3,regular,Tulane,American Athletic,Navy,American Athletic,-7.0,-10.5,1,1,0.243821,NaN,NaN,0.0,2020-09-19 16:00:00+00:00
4,401245593,2020,3,regular,Baylor,Big 12,Houston,American Athletic,-3.5,-0.5,0,0,0.748068,NaN,NaN,0.0,2020-09-19 16:00:00+00:00


In [23]:
predict_df['spread_result'] = np.where(predict_df['spread_target'] <= predict_df['pre_game_spread'], 1, 0)
predict_df.head()

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,regression_spread_pred,classification_spread_pred,classification_confidence,regression_spread_result,classification_spread_result,spread_target,start_date,spread_result
0,401234571,2020,3,regular,Pittsburgh,ACC,Syracuse,ACC,-21.0,-15.0,0,0,0.532774,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,0
1,401234567,2020,3,regular,Duke,ACC,Boston College,ACC,-6.0,2.5,0,1,0.403426,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,0
2,401207124,2020,3,regular,Western Kentucky,Conference USA,Liberty,FBS Independents,-14.0,-6.0,0,0,0.654454,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,0
3,401212522,2020,3,regular,Tulane,American Athletic,Navy,American Athletic,-7.0,-10.5,1,1,0.243821,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,0
4,401245593,2020,3,regular,Baylor,Big 12,Houston,American Athletic,-3.5,-0.5,0,0,0.748068,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,0


In [24]:
# Spread Result Calculation
predict_df['regression_spread_result'] = np.where(predict_df['spread_result'] == predict_df['regression_spread_pred'], 'Won', 'Lost')
predict_df['regression_spread_result'] = np.where(predict_df['spread_target'] == predict_df['pre_game_spread'], 'Push', predict_df['regression_spread_result'])

predict_df['classification_spread_result'] = np.where(predict_df['spread_result'] == predict_df['classification_spread_pred'], 'Won', 'Lost')
predict_df['classification_spread_result'] = np.where(predict_df['spread_target'] == predict_df['pre_game_spread'], 'Push', predict_df['classification_spread_result'])

# Remove Results from Games that haven't been played
predict_df[['regression_spread_result', 'classification_spread_result', 'spread_result']] = predict_df[['regression_spread_result', 'classification_spread_result', 'spread_result']].where(predict_df.spread_target.ne(0), np.nan)

# Rename Column
predict_df = predict_df.rename(columns={'spread_target':'actual_spread'})
predict_df['week'] = predict_df['week'].mode()[0]

predict_df.head()

,game_id,season,week,season_type,home_team,home_conference,away_team,away_conference,pre_game_spread,predicted_spread,regression_spread_pred,classification_spread_pred,classification_confidence,regression_spread_result,classification_spread_result,actual_spread,start_date,spread_result
0,401234571,2020,3,regular,Pittsburgh,ACC,Syracuse,ACC,-21.0,-15.0,0,0,0.532774,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,NaN
1,401234567,2020,3,regular,Duke,ACC,Boston College,ACC,-6.0,2.5,0,1,0.403426,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,NaN
2,401207124,2020,3,regular,Western Kentucky,Conference USA,Liberty,FBS Independents,-14.0,-6.0,0,0,0.654454,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,NaN
3,401212522,2020,3,regular,Tulane,American Athletic,Navy,American Athletic,-7.0,-10.5,1,1,0.243821,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,NaN
4,401245593,2020,3,regular,Baylor,Big 12,Houston,American Athletic,-3.5,-0.5,0,0,0.748068,NaN,NaN,0.0,2020-09-19 16:00:00+00:00,NaN


In [25]:
# Export
filepath = Path('../zillion_picks/picks')

for key, group in predict_df.groupby(['season', 'week', 'season_type']):
    if group.season_type.unique() == 'postseason':
        group.to_csv(filepath/f'premium/{key[0]}_postseason.csv', index=False)
    else:
        group.to_csv(filepath/f'premium/{key[0]}_{str(key[1]).rjust(2, "0")}.csv', index=False)

In [14]:
# REGRESSION AND CLASSIFIER MODEL FOR BACK TESTING

# Load
predict_season = 2019

final_columns = ['game_id', 'season', 'week', 'season_type', 'home_team',
                'home_conference', 'away_team', 'away_conference',  'pre_game_spread',
                'predicted_spread', 'regression_spread_pred', 'classification_spread_pred', 'classification_confidence']

for predict_week in range(1,17):

    df = dataset(predict_season=predict_season, predict_week=predict_week, window_size=4, update_data=False)

    # Prep Dataset
    df[df.columns[df.columns.str.contains('home|Home')]] = df[df.columns[df.columns.str.contains(
        'home|Home')]].groupby('home_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

    df[df.columns[df.columns.str.contains('away|Away')]] = df[df.columns[df.columns.str.contains(
        'away|Away')]].groupby('away_team', as_index=False, group_keys=False).apply(lambda x: x.fillna(x.mean()))

    df = df.dropna(axis=1, thresh=int(len(df) * .9))
    df = df.fillna(0)
    df[df.select_dtypes('bool').columns] = df.select_dtypes('bool').astype('int')

    # Model
    ct = make_column_transformer(
        (OrdinalEncoder(), ['season_type', 'home_conference', 'away_conference']),
        remainder='passthrough'
    )

    drop_columns =  ['game_id', 'season', 'venue', 'start_date', 'home_team', 'away_team', 
                    'start_date', 'spread_target', 'pre_game_spread', 'pre_game_home_win_prob', 'pre_game_away_win_prob']
                    

    # Use in production
    # X_train = df[(df.start_date.dt.tz_convert('US/Central') < today_localize)]
    # X_test = df[(df.start_date.dt.tz_convert('US/Central') >= today_localize)] 

    # Use in development
    X_train = df[(df['week'] < predict_week) | (df['season'] < predict_season)] 
    X_test = df[(df['week'] == predict_week) & (df['season'] == predict_season)] 

    model_regressor = XGBRegressor(learning_rate=0.1, 
                                    colsample_bytree=0.9,
                                    gamma=0.5,
                                    max_depth=2,
                                    min_child_weight=4,
                                    n_estimators=100,
                                    subsample=0.8)

    model_regressor.fit(ct.fit_transform(X_train.drop(drop_columns, axis=1)), X_train['spread_target'])
    
    # Use this for later to determine feature importance
    model_regressor_cols = X_train.drop(drop_columns, axis=1).columns

    X_train['predicted_spread'] = model_regressor.predict(ct.fit_transform(X_train.drop(drop_columns, axis=1)))
    X_test['predicted_spread'] = model_regressor.predict(ct.fit_transform(X_test.drop(drop_columns, axis=1)))

    # Clean Up
    X_train['predicted_spread'] = np.around(X_train['predicted_spread'] / .5, decimals=0) * .5
    X_test['predicted_spread'] = np.around(X_test['predicted_spread'] / .5, decimals=0) * .5

    X_train['regression_spread_pred'] = np.where(X_train['predicted_spread'] <= X_train['pre_game_spread'], 1, 0)
    X_test['regression_spread_pred'] = np.where(X_test['predicted_spread'] <= X_test['pre_game_spread'], 1, 0)

    X_train['spread_result'] = np.where(X_train['spread_target'] <= X_train['pre_game_spread'], 1, 0)
    X_test['spread_result'] = np.where(X_test['spread_target'] <= X_test['pre_game_spread'], 1, 0)

    model_classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                                    colsample_bynode=1, colsample_bytree=0.8, gamma=5, gpu_id=-1,
                                    importance_type='gain', interaction_constraints='',
                                    learning_rate=0.1, max_delta_step=0, max_depth=5,
                                    min_child_weight=10, monotone_constraints='()',
                                    n_estimators=100, n_jobs=0, num_parallel_tree=1,
                                    objective='binary:logistic', random_state=0, reg_alpha=0,
                                    reg_lambda=1, scale_pos_weight=1, subsample=1.0,
                                    tree_method='exact', validate_parameters=1, verbosity=None)

    # Update drop columns list for Classifier Model
    drop_columns.append('spread_result')
    
    # for col in ['pre_game_spread', 'pre_game_home_win_prob', 'pre_game_away_win_prob']:
    #     drop_columns.remove(col)   

    model_classifier.fit(ct.fit_transform(X_train.drop(drop_columns, axis=1)), X_train['spread_result'])
    
    # Use this for later to determine feature importance
    model_classifier_cols = X_train.drop(drop_columns, axis=1).columns

    X_train['classification_spread_pred'] = model_classifier.predict(ct.fit_transform(X_train.drop(drop_columns, axis=1)))
    X_test['classification_spread_pred'] = model_classifier.predict(ct.fit_transform(X_test.drop(drop_columns, axis=1)))

    # Add so the Predict Probability is Aligned
    drop_columns.append('classification_spread_pred')

    X_train['classification_confidence'] = model_classifier.predict_proba(ct.fit_transform(X_train.drop(drop_columns, axis=1)))[:,0]
    X_test['classification_confidence'] = model_classifier.predict_proba(ct.fit_transform(X_test.drop(drop_columns, axis=1)))[:,0] 

    predict_df = X_test[final_columns]

    predict_df = predict_df.merge(df[['game_id', 'spread_target', 'start_date']], on='game_id', how='left')
    predict_df['spread_result'] = np.where(predict_df['spread_target'] <= predict_df['pre_game_spread'], 1, 0)

    predict_df['regression_spread_result'] = np.where(predict_df['spread_result'] == predict_df['regression_spread_pred'], 'Won', 'Lost')
    predict_df['regression_spread_result'] = np.where(predict_df['spread_target'] == predict_df['pre_game_spread'], 'Push', predict_df['regression_spread_result'])

    predict_df['classification_spread_result'] = np.where(predict_df['spread_result'] == predict_df['classification_spread_pred'], 'Won', 'Lost')
    predict_df['classification_spread_result'] = np.where(predict_df['spread_target'] == predict_df['pre_game_spread'], 'Push', predict_df['classification_spread_result'])

    # Remove Results from Games that haven't been played
    predict_df[['regression_spread_result', 'classification_spread_result', 'spread_result']] = predict_df[['regression_spread_result', 'classification_spread_result', 'spread_result']].where(predict_df.spread_target.ne(0), np.nan)

    # Rename Column
    predict_df = predict_df.rename(columns={'spread_target':'actual_spread'})
    predict_df['week'] = predict_df['week'].mode()[0]

    # Export
    filepath = Path('../zillion_picks/picks')

    for key, group in predict_df.groupby(['season', 'week', 'season_type']):
        if group.season_type.unique() == 'postseason':
            group.to_csv(filepath/f'premium/{key[0]}_postseason.csv', index=False)
        else:
            group.to_csv(filepath/f'premium/{key[0]}_{str(key[1]).rjust(2, "0")}.csv', index=False)

    print(f"{predict_season} Week {predict_week} Regression Accuracy: {round(accuracy_score(X_test['spread_result'], X_test['regression_spread_pred']) * 100,2)}%" )
    print(f"{predict_season} Week {predict_week} Classification Accuracy: {round(accuracy_score(X_test['spread_result'], X_test['classification_spread_pred']) * 100,2)}%" )
    print('--------------')


2019 Week 1 Regression Accuracy: 55.42%
2019 Week 1 Classification Accuracy: 60.24%
--------------
2019 Week 2 Regression Accuracy: 54.79%
2019 Week 2 Classification Accuracy: 45.21%
--------------
2019 Week 3 Regression Accuracy: 41.79%
2019 Week 3 Classification Accuracy: 55.22%
--------------
2019 Week 4 Regression Accuracy: 46.55%
2019 Week 4 Classification Accuracy: 56.9%
--------------
2019 Week 5 Regression Accuracy: 60.0%
2019 Week 5 Classification Accuracy: 47.27%
--------------
2019 Week 6 Regression Accuracy: 57.45%
2019 Week 6 Classification Accuracy: 61.7%
--------------
2019 Week 7 Regression Accuracy: 49.06%
2019 Week 7 Classification Accuracy: 56.6%
--------------
2019 Week 8 Regression Accuracy: 50.82%
2019 Week 8 Classification Accuracy: 50.82%
--------------
2019 Week 9 Regression Accuracy: 54.55%
2019 Week 9 Classification Accuracy: 58.18%
--------------
2019 Week 10 Regression Accuracy: 52.08%
2019 Week 10 Classification Accuracy: 56.25%
--------------
2019 Week 11

In [15]:
# Test Feature Importance

regression_important_features = pd.Series(data=model_regressor.feature_importances_, index=model_regressor_cols).sort_values(ascending=False)
classification_important_features = pd.Series(data=model_classifier.feature_importances_, index=model_classifier_cols).sort_values(ascending=False)

print(regression_important_features.head(10))
print('---------------------------------------')
print(classification_important_features.head(10))


pointsAway                          0.058916
away_points                         0.040906
offense_successRateHome             0.037539
averageRating_All PositionsHome     0.031255
rankHome                            0.029669
averageRating_Defensive LineHome    0.025599
home_conference                     0.024919
averageStars_All PositionsHome      0.023821
talentHome                          0.021426
home_points                         0.021078
dtype: float32
---------------------------------------
regression_spread_pred            0.010797
averageStars_All PositionsHome    0.007093
year WRHome                       0.006112
away_points                       0.005852
possessionTimeHome                0.005155
weight RBHome                     0.005017
totalRating_All PositionsHome     0.005001
rushingTDsHome                    0.004950
defense_lineYardsHome             0.004919
completionAttemptsSuccessAway     0.004907
dtype: float32


In [9]:
    print(f"{predict_season} Week {predict_week} Classification Accuracy: {round(accuracy_score(X_train['spread_result'], X_train['classification_spread_pred']) * 100,2)}%" )

2019 Week 16 Classification Accuracy: 92.34%


In [535]:
#################### WORK ON THIS ###############################

# CHECK CONFIDENCE TO SEE IF THERE IS A WIN ADVANTAGE
dfs=[]

for file in (filepath/'free').rglob('*'):
    df = pd.read_csv(file)
    dfs.append(df)

df = pd.concat(dfs)
df.head()

df['bin'] = pd.cut(df.classification_confidence, [0,.20,.35,.40,.45,.50,.55,.60,.65,.70,.80,1])
df.groupby('bin')['classification_spread_result'].agg(win=(lambda df: df.eq('Won').sum() / len(df)), count=( 'count'))

,win,count
bin,,
"(0.0, 0.2]",0.750000,4
"(0.2, 0.35]",0.466667,90
"(0.35, 0.4]",0.511364,88
"(0.4, 0.45]",0.521739,115
"(0.45, 0.5]",0.535088,114
"(0.5, 0.55]",0.513333,150
"(0.55, 0.6]",0.507692,130
"(0.6, 0.65]",0.518072,83
"(0.65, 0.7]",0.561404,57
